In [1]:
import os
import sys
import argparse
from typing import List

import torch
import transformers
from utils import *
from datasets import load_dataset, load_from_disk, DatasetDict

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from Prompter import Prompter, ZeroPrompter

device = "cuda" if torch.cuda.is_available() else "cpu"

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Ноутбук с заготовкой LORA

In [2]:
####### PARAMS #######

MODEL_PRUNE = '../cache/Whitening_model_test.pt'
DATA_PATH = 'python_code_instructions_18k_alpaca_ru'
OUTPUT_DIR = '../cache/Whitening_model_test_res/'
# MODEL_PRUNE = '../cache/jeffwan_llama_7b_hf_whitening_only_0.8.pt'
# DATA_PATH = 'python_code_instructions_18k_alpaca_ru'
# OUTPUT_DIR = '../cache/jeffwan_llama_7b_hf_whitening_only_0.8/'
LORA_R = 8 # Rank of lora 
LORA_ALPHA = 16
NUM_EPOCHS = 2 # Number of fine tune epochs
LEARNING_RATE = 1e-4
BATCH_SIZE = 64
MICRO_BATCH_SIZE = 4 # number of different forwards before accumulation grads in each batch 
CUTOFF_LENGTH = 400
TRAIN_ON_INPUTS = False # Should model learn on input text also or not
LORA_TARGET_MODULES = "q_v_proj,q_u_proj,k_v_proj,k_u_proj,v_u_proj,\
v_v_proj,o_u_proj,o_v_proj,gate_u_proj,gate_v_proj,down_u_proj,down_v_proj,up_u_proj,up_v_proj,\
q_proj,k_proj,v_proj,o_proj,gate_proj,up_proj,down_proj"
LORA_DROPOUT = 0.05
VAL_SIZE = 10
EVAL_STEPS = 0

In [3]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model = AutoModelForCausalLM.from_pretrained('../cache/Witening_SVD_0.3_postrain/')
# tokenizer = AutoTokenizer.from_pretrained('../cache/Witening_SVD_0.3_postrain_Tokenizer/')

In [ ]:
# Load Pruned Model

pruned_dict = torch.load(MODEL_PRUNE, map_location='cpu')
tokenizer, model = pruned_dict['tokenizer'], pruned_dict['model']
gradient_accumulation_steps = BATCH_SIZE // MICRO_BATCH_SIZE

prompter = Prompter('alpaca')

if device == 'cuda':
    model.half()

tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LENGTH,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LENGTH
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if TRAIN_ON_INPUTS:
        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=False
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if False:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    return tokenized_full_prompt

def split_and_tokenizer(test_data, tokenizer, seq_len, field_name):
    test_ids = tokenizer("\n\n".join(test_data[field_name]), return_tensors='pt').input_ids[0]
    test_ids_batch = []
    nsamples = test_ids.numel() // seq_len

    test_set = []
    for i in range(nsamples):
        batch = test_ids[(i * seq_len):((i + 1) * seq_len)]
        test_set.append({
            'input_ids': batch,
            'labels': batch
        })
    return test_set

# Prepare For LoRA
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES.split(","),
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()  

# Load Train Dataset
try:
    data = load_dataset(DATA_PATH)
except:
    data = DatasetDict({'train':load_from_disk(DATA_PATH)})
    
train_val = data["train"].train_test_split(
    test_size=VAL_SIZE, shuffle=True, seed=42
)
train_data = (
    train_val["train"].shuffle().map(generate_and_tokenize_prompt)
)
val_data = {
    DATA_PATH: train_val["test"].shuffle().map(generate_and_tokenize_prompt),
}

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=10,
        logging_first_step=True,
        optim="adamw_torch",
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=EVAL_STEPS,
        save_steps=200,
        output_dir=OUTPUT_DIR,
        save_total_limit=20,
        load_best_model_at_end=True,
        ddp_find_unused_parameters=None,
        group_by_length=False,
        report_to="none",
        run_name="none",
        metric_for_best_model="{}_loss".format(DATA_PATH),
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)
model.config.use_cache = False
old_state_dict = model.state_dict

trainer.train(resume_from_checkpoint=False)

model.state_dict = old_state_dict
model.save_pretrained(OUTPUT_DIR)

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/peft/utils/other.py:135: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 21,743,104 || all params: 1,580,758,528 || trainable%: 1.37548547832348


Map:   0%|          | 0/9990 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

comet_ml is installed but `COMET_API_KEY` is not set.
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 5.4.270, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Python Code Instructions 18k Alpaca Ru Loss
10,8.791900,No log,8.238528
20,8.048800,No log,6.882577
30,6.183000,No log,4.943671
40,4.528700,No log,3.706689
50,3.565100,No log,2.998249
60,2.956200,No log,2.424523
70,2.307200,No log,1.899150
80,1.877600,No log,1.544174
90,1.569400,No log,1.324146
100,1.375200,No log,1.201641


In [8]:
!pip install transformers==4.35.2
# !pip install accelerate --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, pipeline, TextIteratorStreamer

pruned_dict = torch.load(MODEL_PRUNE, map_location='cpu')
tokenizer, model = pruned_dict['tokenizer'], pruned_dict['model']
model = PeftModel.from_pretrained(model, OUTPUT_DIR, device_map={"":0}).half()
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): SVD_LlamaAttention(
              (q_u_proj): Linear(
                in_features=409, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=409, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (q_v_proj): Linear(
                in_features=4096, out_features=409, bias=False
                (l

In [7]:
model.to('cuda')

input_ids = tokenizer(["Сгенерируй код сортировки пузырьком на Python"], return_tensors="pt", truncation=True).input_ids.cuda()
with torch.inference_mode():
    outputs = model.generate(input_ids=input_ids, max_new_tokens=200, do_sample=True, top_p=0.9)

print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


<｜begin▁of▁sentence｜>Сгенерируй код сортировки пузырьком на Python, чтобы оптимизировать этот код.

def quicksort(l):
    l = l[:-1]
    for i in range(len(l)-1):
            if l[i] > l[i] + l[i]**2, l[i]=l[i] + l[i]**2**2]
    l[i] = l[i-1]

# Output
lquicksort(list)<|EOT|>
